In [245]:
%load_ext autoreload
%autoreload 2

from IPython.display import display

import pandas as pd
import numpy as np 
import CBOEFileParser
import CBOE_Downloader
import POP
import VerticalSpreads
import IronCondors

from pymongo import MongoClient




import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
pop_calculator = POP.POP().get_ohlc_data(underlying_symbol)

In [306]:
%%capture

underlying_symbol = "XLU"
risk_free_rate = 2.98

# cboe_file = CBOE_Downloader.CBOE_Downloader(underlying_symbol).download().save_download().get_filename()
# cboe_file = "/Users/glenn/Documents/CBOE_Data/BIDU_201807261013.dat"
# cboe_file = "/Users/glenn/Documents/CBOE_Data/GOOG_201807261202.dat"
cboe_file = "/Users/glenn/Documents/CBOE_Data/GOOG_201807301217.dat"



option_chains = CBOEFileParser.CBOEFileParser() \
                    .parse_cboe_file(cboe_file) \
                    .remove_options_after(days=60) \
                    .remove_options_before(days=30) \
                    .set_exchange() \
                    .add_greeks(risk_free_rate=risk_free_rate)

In [308]:
iron_condors_obj = IronCondors.IronCondors() \
                            .build_condors(option_chains, risk_limit=200) \
                            .itm_condors(option_chains.underlying_spot) \
                            .sky_view_condors()
#                             .add_pop(pop_calculator)

print(option_chains.underlying_symbol)
print("Current Spot: {}".format(option_chains.underlying_spot))

There are 13,530 bull put spreads
There are 13,410 bear call spreads
There are 181,437,300 condors for expiration 2018-09-21 16:00:00
There are 6,903 bull put spreads
There are 6,742 bear call spreads
There are 46,540,026 condors for expiration 2018-08-31 16:00:00
There are 3,486 bull put spreads
There are 3,370 bear call spreads
There are 11,747,820 condors for expiration 2018-09-07 16:00:00
GOOG
Current Spot: 1224.99


In [305]:

client = MongoClient("mongodb://localhost:27017")


total_condors = 0

for expiration,v in iron_condors_obj.condors.items():
    iron_condors = v["iron condors"]
    bull_condors = v["bull condors"]
    bear_condors = v["bear condors"]
    print("Expiration: {}".format(expiration))
    print("Iron Condors: {}".format(len(iron_condors)))
    print("Bull Condors: {}".format(len(bull_condors)))    
    print("Bear Condors: {}".format(len(bear_condors)))
    
    total_condors += len(iron_condors)
    total_condors += len(bull_condors)
    total_condors += len(bear_condors)



    
    for iron_condor in iron_condors:
        condor_data = iron_condor.to_dict()
        condor_data['symbol'] = option_chains.underlying_symbol
        condor_data['timestamp'] = option_chains.timestamp
        condor_data['expiration'] = expiration
        condor_data['type'] = "iron condor"
        
        client.Spreads.condors.insert_one(condor_data)
    
    for bull_condor in bull_condors:
        condor_data = bull_condor.to_dict()
        condor_data['symbol'] = option_chains.underlying_symbol
        condor_data['timestamp'] = option_chains.timestamp
        condor_data['expiration'] = expiration
        condor_data['type'] = "bull condor"
        
        client.Spreads.condors.insert_one(condor_data)
        
    for bear_condor in bear_condors:
        condor_data = bear_condor.to_dict()
        condor_data['symbol'] = option_chains.underlying_symbol
        condor_data['timestamp'] = option_chains.timestamp
        condor_data['expiration'] = expiration
        condor_data['type'] = "bear condor"

        client.Spreads.condors.insert_one(condor_data)
        
    
print("Total Number of condors found: {:,}".format(total_condors))

Expiration: 2018-09-21 16:00:00
Iron Condors: 112
Bull Condors: 1
Bear Condors: 6
Expiration: 2018-08-31 16:00:00
Iron Condors: 2133
Bull Condors: 89
Bear Condors: 339
Expiration: 2018-09-07 16:00:00
Iron Condors: 2460
Bull Condors: 117
Bear Condors: 395
Total Number of condors found: 5,652


In [253]:
class VerticalSpread:
    def __init__(self, spread_data_df):
        self.K1_Ask = spread_data_df["K1_Ask"]
        self.K1_Bid = spread_data_df["K1_Bid"]
        self.K1_Description = spread_data_df["K1_Description"]        
        self.K1_Exchange = spread_data_df["K1_Exchange"]        
        self.K1_Last_Sale = spread_data_df["K1_Last Sale"]        
        self.K1_Net = spread_data_df["K1_Net"]        
        self.K1_Open_Int = spread_data_df["K1_Open Int"]        
        self.K1_Strike = spread_data_df["K1_Strike"]        
        self.K1_Vol = spread_data_df["K1_Vol"]
        self.K1_Sigma = spread_data_df["K1_Sigma"]                
        self.K1_Delta = spread_data_df["K1_Delta"]                
        self.K1_Gamma = spread_data_df["K1_Gamma"]                
        self.K1_Theta = spread_data_df["K1_Theta"]                
        self.K1_Vega = spread_data_df["K1_Vega"]                
        self.K1_Entry = spread_data_df["K1_Entry"]
        
        self.K2_Ask = spread_data_df["K2_Ask"]
        self.K2_Bid = spread_data_df["K2_Bid"]
        self.K2_Description = spread_data_df["K2_Description"]        
        self.K2_Exchange = spread_data_df["K2_Exchange"]        
        self.K2_Last_Sale = spread_data_df["K2_Last Sale"]        
        self.K2_Net = spread_data_df["K2_Net"]        
        self.K2_Open_Int = spread_data_df["K2_Open Int"]        
        self.K2_Strike = spread_data_df["K2_Strike"]        
        self.K2_Vol = spread_data_df["K2_Vol"]
        self.K2_Sigma = spread_data_df["K2_Sigma"]                
        self.K2_Delta = spread_data_df["K2_Delta"]                
        self.K2_Gamma = spread_data_df["K2_Gamma"]                
        self.K2_Theta = spread_data_df["K2_Theta"]                
        self.K2_Vega = spread_data_df["K2_Vega"]                
        self.K2_Entry = spread_data_df["K2_Entry"]
        
        self.max_profit = spread_data_df["Max Profit"]
        self.max_risk = spread_data_df["Max Risk"]
        self.break_even = spread_data_df["Break Even"]
        self.delta = spread_data_df["Delta"]
        self.gamma = spread_data_df["Gamma"]
        self.theta = spread_data_df["Theta"]
        self.vega = spread_data_df["Vega"]

In [193]:
class IronCondor:
    def __init__(self, bull_side, bear_side):
        self.K1_Ask = bull_side.K1_Ask
        self.K1_Bid = bull_side.K1_Bid
        self.K1_Description = bull_side.K1_Description        
        self.K1_Exchange = bull_side.K1_Exchange        
        self.K1_Last_Sale = bull_side.K1_Last_Sale      
        self.K1_Net = bull_side.K1_Net        
        self.K1_Open_Int = bull_side.K1_Open_Int        
        self.K1_Strike = bull_side.K1_Strike        
        self.K1_Vol = bull_side.K1_Vol
        self.K1_Sigma = bull_side.K1_Sigma                
        self.K1_Delta = bull_side.K1_Delta                
        self.K1_Gamma = bull_side.K1_Gamma                
        self.K1_Theta = bull_side.K1_Theta                
        self.K1_Vega = bull_side.K1_Vega                
        self.K1_Entry = bull_side.K1_Entry
        
        self.K2_Ask = bull_side.K2_Ask
        self.K2_Bid = bull_side.K2_Bid
        self.K2_Description = bull_side.K2_Description        
        self.K2_Exchange = bull_side.K2_Exchange        
        self.K2_Last_Sale = bull_side.K2_Last_Sale      
        self.K2_Net = bull_side.K2_Net        
        self.K2_Open_Int = bull_side.K2_Open_Int        
        self.K2_Strike = bull_side.K2_Strike        
        self.K2_Vol = bull_side.K2_Vol
        self.K2_Sigma = bull_side.K2_Sigma                
        self.K2_Delta = bull_side.K2_Delta                
        self.K2_Gamma = bull_side.K2_Gamma                
        self.K2_Theta = bull_side.K2_Theta                
        self.K2_Vega = bull_side.K2_Vega                
        self.K2_Entry = bull_side.K2_Entry
        
        self.K3_Ask = bear_side.K1_Ask
        self.K3_Bid = bear_side.K1_Bid
        self.K3_Description = bear_side.K1_Description        
        self.K3_Exchange = bear_side.K1_Exchange        
        self.K3_Last_Sale = bear_side.K1_Last_Sale      
        self.K3_Net = bear_side.K1_Net        
        self.K3_Open_Int = bear_side.K1_Open_Int        
        self.K3_Strike = bear_side.K1_Strike        
        self.K3_Vol = bear_side.K1_Vol
        self.K3_Sigma = bear_side.K1_Sigma                
        self.K3_Delta = bear_side.K1_Delta                
        self.K3_Gamma = bear_side.K1_Gamma                
        self.K3_Theta = bear_side.K1_Theta                
        self.K3_Vega = bear_side.K1_Vega                
        self.K3_Entry = bear_side.K1_Entry
        
        self.K4_Ask = bear_side.K2_Ask
        self.K4_Bid = bear_side.K2_Bid
        self.K4_Description = bear_side.K2_Description        
        self.K4_Exchange = bear_side.K2_Exchange        
        self.K4_Last_Sale = bear_side.K2_Last_Sale      
        self.K4_Net = bear_side.K2_Net        
        self.K4_Open_Int = bear_side.K2_Open_Int        
        self.K4_Strike = bear_side.K2_Strike        
        self.K4_Vol = bear_side.K2_Vol
        self.K4_Sigma = bear_side.K2_Sigma                
        self.K4_Delta = bear_side.K2_Delta                
        self.K4_Gamma = bear_side.K2_Gamma                
        self.K4_Theta = bear_side.K2_Theta                
        self.K4_Vega = bear_side.K2_Vega                
        self.K4_Entry = bear_side.K2_Entry
        
        self.max_profit = self.K1_Entry + self.K2_Entry + self.K3_Entry + self.K4_Entry
        self.max_risk = self.get_max_risk()
        self.put_break_even = self.get_break_even(strike_diff=(self.K2_Strike-self.K1_Strike), low_strike=self.K1_Strike)
        self.call_break_even = self.get_break_even(strike_diff=(self.K4_Strike-self.K3_Strike), low_strike=self.K3_Strike)
        self.delta = self.K1_Delta + self.K2_Delta + self.K3_Delta + self.K4_Delta
        self.gamma = self.K1_Gamma + self.K2_Gamma + self.K3_Gamma + self.K4_Gamma
        self.theta = self.K1_Theta + self.K2_Theta + self.K3_Theta + self.K4_Theta
        self.vega = self.K1_Vega + self.K2_Vega + self.K3_Vega + self.K4_Vega
        
        if (self.put_break_even==None) or (self.call_break_even==None):
            self.broken_condor = True
            self.adjust_max_profit()
        else:
            self.broken_condor = False
            
        
    def get_max_risk(self):
        put_strike_diff = self.K2_Strike - self.K1_Strike
        call_strike_diff = self.K4_Strike - self.K3_Strike
        if put_strike_diff > call_strike_diff:
            max_risk = -put_strike_diff + self.max_profit
        else:
            max_risk = -call_strike_diff + self.max_profit
            
        return max_risk
    
    def get_break_even(self, strike_diff, low_strike):
        # check for a broken wing condor:
        if self.max_profit > strike_diff:
            break_even = None
        else:
            break_even = low_strike + self.max_profit
            
        return break_even
            
    def adjust_max_profit(self):
        if self.put_break_even == None:
            self.max_profit -= (self.K2_Strike - self.K1_Strike)
        
        if self.call_break_even == None:
            self.max_profit -= (self.K4_Strike - self.K3_Strike)
        
    def valid_condor(self):
        max_profit_ok = False
        max_risk_ok = False
        break_evens_ok = False
        put_break_even_ok = False
        call_break_even_ok = False
        
        if self.max_profit > 0:
            max_profit_ok = True
        
        if self.max_risk < 0:
            max_risk_ok = True
            
        if self.put_break_even or self.call_break_even:
            break_evens_ok = True
        
        if (self.put_break_even==None) or (self.K1_Strike < self.put_break_even < self.K2_Strike):
            put_break_even_ok = True
        
        if (self.call_break_even==None) or (self.K3_Strike < self.call_break_even < self.K4_Strike):
            call_break_even_ok = True
        
            
        if (max_profit_ok & max_risk_ok & break_evens_ok & put_break_even_ok & call_break_even_ok):
            return self
        else:
            return None
            

In [209]:
# Expiration: 2018-08-31 16:00:00
expiration, v = list(iron_condors_obj.vertical_spreads.items())[1]
bull_puts_df = v["Bull Puts"]
bear_calls_df = v["Bear Calls"]
bull_puts = [VerticalSpread(bull_puts_df.iloc[idx]) for idx in range(bull_puts_df.shape[0])]
bear_calls = [VerticalSpread(bear_calls_df.iloc[idx]) for idx in range(bear_calls_df.shape[0])]
print("Expiration: {}".format(expiration))
print("bull_puts.shape: {}".format(bull_puts_df.shape))
print("bear_calls.shape: {}".format(bear_calls_df.shape))
print("max possible condors: {:,}".format(bull_puts_df.shape[0]*bear_calls_df.shape[0]))

Expiration: 2018-08-31 16:00:00
bull_puts.shape: (1081, 38)
bear_calls.shape: (1052, 38)
max possible condors: 1,137,212


In [211]:
start_time = datetime.datetime.today()

# This ran in 12s
# Total condors: 852,390
# iron_condors = [IronCondor(bull_put, bear_call) for bull_put in bull_puts for bear_call in bear_calls]

# This ran in 1.7s
# Total condors: 102,087
condors = [IronCondor(bull_put, bear_call) for bull_put in bull_puts for bear_call in bear_calls if (bull_put.K2_Strike<=bear_call.K1_Strike) and (bull_put.K1_Strike<option_chains.underlying_spot<bear_call.K2_Strike)]

end_time = datetime.datetime.today()

elapsed_time = end_time - start_time
print("Elapsed time: {}".format(elapsed_time))
print("Total Iron Condors: {:,}".format(len(condors)))

Elapsed time: 0:00:01.910238
Total Iron Condors: 153,121


In [212]:
# check if they are valid
valid_condors = [condor for condor in condors if condor.valid_condor()]

# risk limit
risk_limit = 500
risk_limit = -risk_limit/100.0
valid_condors = [condor for condor in valid_condors if condor.max_risk > risk_limit]

# separate the condors into:
iron_condors = [condor for condor in valid_condors if (condor.put_break_even and condor.call_break_even)]
bull_condors = [condor for condor in valid_condors if (condor.put_break_even and condor.call_break_even==None)]
bear_condors = [condor for condor in valid_condors if (condor.put_break_even==None and condor.call_break_even)]


# ITM condors:
bull_condors = [condor for condor in bull_condors if (condor.put_break_even<option_chains.underlying_spot)]
bear_condors = [condor for condor in bear_condors if (condor.call_break_even>option_chains.underlying_spot)]
iron_condors = [condor for condor in iron_condors if (condor.put_break_even<option_chains.underlying_spot<condor.call_break_even)]

# Sky View Criteria:
iron_condors1 = [condor for condor in iron_condors if (((condor.K2_Strike-condor.K1_Strike) >=(condor.K4_Strike-condor.K3_Strike)) and (condor.K2_Strike-condor.K1_Strike)*0.2<=condor.max_profit<=(condor.K2_Strike-condor.K1_Strike)*0.33)]
iron_condors2 = [condor for condor in iron_condors if (((condor.K2_Strike-condor.K1_Strike) < (condor.K4_Strike-condor.K3_Strike)) and (condor.K4_Strike-condor.K3_Strike)*0.2<=condor.max_profit<=(condor.K4_Strike-condor.K3_Strike)*0.33)]
iron_condors = iron_condors1 + iron_condors2

bull_condors = [condor for condor in bull_condors if ((condor.K2_Strike-condor.K1_Strike)*0.2<=condor.max_profit<=(condor.K2_Strike-condor.K1_Strike)*0.33)]
bear_condors = [condor for condor in bear_condors if ((condor.K4_Strike-condor.K3_Strike)*0.2<=condor.max_profit<=(condor.K4_Strike-condor.K3_Strike)*0.33)]

# Add P.O.P.:



print("There are {} Iron Condors".format(len(iron_condors)))
print("There are {} Bull Condors".format(len(bull_condors)))
print("There are {} Bear Condors".format(len(bear_condors)))

There are 312 Iron Condors
There are 19 Bull Condors
There are 12 Bear Condors


In [205]:
[1, 2, 3] + [5, 6, 7]

[1, 2, 3, 5, 6, 7]

In [140]:
# this_condor = valid_condors[0]
# this_condor = broke_wing_condors[0]
# this_condor = bull_condors[0]
# this_condor = bear_condors[2]
print("Strikes: {}/{}/{}/{}".format(this_condor.K1_Strike,this_condor.K2_Strike,this_condor.K3_Strike,this_condor.K4_Strike))
print("Max Risk: {}".format(this_condor.max_risk))
print("Max Profit: {}".format(this_condor.max_profit))
print("Put Break Even: {}".format(this_condor.put_break_even))
print("Call Break Even: {}".format(this_condor.call_break_even))
print("Underlying Spot: {}".format(option_chains.underlying_spot))

Strikes: 250.0/257.5/257.5/260.0
Max Risk: 0.5
Max Profit: 0.5
Put Break Even: 254.5
Call Break Even: None
Underlying Spot: 259.54


In [123]:
print("There are {} Bull Condors".format(len(bull_condors)))
bull_condors = [bull_condor for bull_condor in bull_condors if (bull_condor.put_break_even<option_chains.underlying_spot)]
bull_condors = [bull_condor for bull_condor in bull_condors if (bull_condor.K1_Strike<bull_condor.put_break_even<bull_condor.K2_Strike)]
bull_condors = [bull_condor for bull_condor in bull_condors if (bull_condor.max_risk<0)]
print("There are {} Bull Condors".format(len(bull_condors)))

# 250.0/257.5/257.5/260.0
# Max Risk: -3.50
# Max Profit: 1.50
# Put Break Even: 253.50
# Call Break Even: NaN
# P.O.P: 1.0
# Delta: 1.9171
# Gamma: 0.0048
# Vega: -3.0677
# Theta: 0.1609









There are 600 Bull Condors
There are 0 Bull Condors


In [199]:
this_condor = None
# for condor in broke_wing_condors:
# for condor in bull_condors:
for condor in bear_condors:
    if condor.K1_Strike==255.0 and condor.K2_Strike==257.5 and condor.K3_Strike==257.5 and condor.K4_Strike==267.5:
        print("Found it!!!")
        this_condor = condor
        

Found it!!!


In [200]:
print("Strikes: {}/{}/{}/{}".format(this_condor.K1_Strike,this_condor.K2_Strike,this_condor.K3_Strike,this_condor.K4_Strike))
print("Max Risk: {}".format(this_condor.max_risk))
print("Max Profit: {}".format(this_condor.max_profit))
print("Put Break Even: {}".format(this_condor.put_break_even))
print("Call Break Even: {}".format(this_condor.call_break_even))
print("K1_Entry: {}".format(this_condor.K1_Entry))
print("K2_Entry: {}".format(this_condor.K2_Entry))
print("K3_Entry: {}".format(this_condor.K3_Entry))
print("K4_Entry: {}".format(this_condor.K4_Entry))


Strikes: 255.0/257.5/257.5/267.5
Max Risk: -4.849999999999998
Max Profit: 2.650000000000002
Put Break Even: None
Call Break Even: 262.65
K1_Entry: -9.85
K2_Entry: 10.8
K3_Entry: 13.3
K4_Entry: -9.1


In [192]:
max_profit = -9.85 + 10.8 + 13.30 -9.1
print("initial max profit: {}".format(max_profit))
print("break even: {}".format(257.5 + max_profit))
print("max risk: {}".format(257.5-267.5+max_profit))
print("max profit: {}".format(max_profit-(257.5-255)))



initial max profit: 5.150000000000002
break even: 262.65
max risk: -4.849999999999998
max profit: 2.650000000000002


In [154]:
# Strike Display: 255.0/257.5/257.5/267.5
# Max Profit: 2.65
# Max Risk: -4.85
# Put Break Even: NaN
# Call Break Even: 262.65
# Delta: 1.907208
# Gamma: 0.005447
# Theta: -3.100194
# Vega: 0.178153
# POP: 0.0

# K1_Entry: -9.85
# K2_Entry: 10.8
# K3_Entry: 13.30
# K4_Entry: -9.1

# K1_Ask: 9.85
# K1_Bid: 9.6
# K1_Description: BIDU1831T255
# K1_Exchange: None
# K1_Last Sale: 8.15
# K1_Net: 0.0
# K1_Open Int: 74
# K1_Strike: 255.0
# K1_Vol: 0
# K1_Sigma: 0.471549
# K1_Delta: -0.014467
# K1_Gamma: 0.000952
# K1_Theta: 0.012813
# K1_Vega: 0.030022
# K2_Ask: 11.05
# K2_Bid: 10.8
# K2_Description: BIDU1831T257.5
# K2_Exchange: None
# K2_Last Sale: 9.65
# K2_Net: 0.0
# K2_Open Int: 133
# K2_Strike: 257.5
# K2_Vol: 0
# K2_Sigma: 0.483129
# K2_Delta: -0.019154
# K2_Gamma: 0.001181
# K2_Theta: 0.017521
# K2_Vega: 0.038170

# K3_Ask: 13.60
# K3_Bid: 13.30
# K3_Description: BIDU1831H257.5
# K3_Exchange: None
# K3_Last Sale: 14.77
# K3_Net: -0.83
# K3_Open Int: 36
# K3_Strike: 257.5
# K3_Vol: 3
# K3_Sigma: 0.524989
# K3_Delta: 0.972510
# K3_Gamma: 0.001474
# K3_Theta: -1.539184
# K3_Vega: 0.051746
# K4_Ask: 9.1
# K4_Bid: 8.75
# K4_Description: BIDU1831H267.5
# K4_Exchange: None
# K4_Last Sale: 11.00
# K4_Net: 0.0
# K4_Open Int: 22
# K4_Strike: 267.5
# K4_Vol: 0
# K4_Sigma: 0.473117
# K4_Delta: 0.968318
# K4_Gamma: 0.001840
# K4_Theta: -1.591343
# K4_Vega: 0.058216
# K3_Entry: 13.30
# K4_Entry: -9.1
